# Operation Safe Passage: Map Creation

This notebook demonstrates how to generate a hexagonal map using the OSP `MapGenerator`.

The process includes:
- Defining a minimal configuration file (`params.json`)
- Explaining each required parameter
- Generating the map and exporting it to JSON
- Inspecting the structure of the output
- Visualizing the graph

In [ ]:
# Standard libraries for filesystem operations and JSON handling
import json

# Import the MapGenerator class from the package
from operation_safe_passage.environment.map_generator import MapGenerator


## Minimal Configuration

The `MapGenerator` requires the `environment` section of `params.json`.

Required parameters:
- `num_nodes`: total number of grid cells (larger values produce larger maps).
- `mine_likelihood`: probability that a given node contains a mine.
- `terrain_types`: list of possible terrain categories.
- `transition_matrix`: probability matrix controlling adjacency of terrains.
- `precipitation_params`: rain likelihood and intensity per terrain.
- `temp_params`: temperature baseline and variation per terrain.
- `wind_params`: wind baseline and variation per terrain.
- `visibility_params`: maximum visibility and reductions due to time or precipitation.

In [ ]:
# Minimal configuration containing only the required parameters (presaved in config/params_map.json)
minimal_config = {
    "environment": {
        "num_nodes": 200,             # Number of hexagonal cells
        "mine_likelihood": 0.1,       # Fraction of nodes seeded with mines
        "terrain_types": ["Grassy", "Rocky"],  # Available terrain categories
        "transition_matrix": [        # Terrain adjacency probabilities
            [0.7, 0.3],               # From Grassy -> 70% Grassy, 30% Rocky
            [0.4, 0.6]                # From Rocky -> 40% Grassy, 60% Rocky
        ],
        "precipitation_params": {     # Rainfall settings
            "Grassy": {"rain_probability": 0.1, "scale": 10},
            "Rocky": {"rain_probability": 0.05, "scale": 5}
        },
        "temp_params": {              # Temperature variation per terrain
            "Grassy": {"base_temp": 20, "diurnal_variation": 10, "precipitation_effect": 0.1},
            "Rocky": {"base_temp": 15, "diurnal_variation": 8, "precipitation_effect": 0.1}
        },
        "wind_params": {              # Wind variation per terrain
            "Grassy": {"base_wind": 10, "time_variation": 5, "precipitation_effect": 0.05},
            "Rocky": {"base_wind": 15, "time_variation": 4, "precipitation_effect": 0.1}
        },
        "visibility_params": {        # Visibility variation per terrain
            "Grassy": {"max_visibility": 100, "time_effect": 5, "precipitation_effect": 2},
            "Rocky": {"max_visibility": 100, "time_effect": 6, "precipitation_effect": 2.5}
        }
    }
}

## The `MapGenerator` Class

`MapGenerator` is the component that constructs the **hexagonal grid map** used throughout Operation Safe Passage.  
When initialized, it loads parameters from a configuration file and prepares everything needed to create and export a map.

**Inputs to `MapGenerator`:**
- `parameters_file` *(str)*: Path to a JSON file containing the `environment` configuration.  

- `output_dir` *(str, optional)*: Folder where generated files will be written.  
  If not provided, defaults to `./output`.  

- `json_filename` *(str, optional)*: Name of the file where the generated network is saved.  
  Default is `"network.json"`.  

**What happens when instantiated:**
- Parameters are loaded into memory from the JSON file.  
- Internal constants are set:
  - The six hexagonal neighbor directions.  
  - Utility functions for generating environmental attributes (precipitation, temperature, wind, visibility).  
- The output directory is created if it does not already exist.  

At this point, the generator is configured, but the map is not created until we call `gen.run()`.


In [ ]:
# Initialize the MapGenerator with the saved configuration
gen = MapGenerator(parameters_file="config/params_map.json", output_dir="output")

# Generate the map and export to output/network.json
gen.run()

# Load the generated JSON for inspection
with open("output/network.json") as f:
    network = json.load(f)

# Display basic properties of the generated map
print("Start node:", network["mission"]["start"])
print("End node:", network["mission"]["end"])
print("Total nodes:", len(network["nodes"]))
print("Sample node with metadata:")
print(json.dumps(network["nodes"][0], indent=2))


## Visualizing the Map

The `network.json` file can be loaded into an `Environment` object.  
This class manages the map structure and provides helper utilities such as visualization.

The built-in `visualize_network` method:
- Infers positions of nodes from their grid coordinates
- Colors nodes based on terrain type
- Draws black outlines for nodes containing mines
- Marks the **start node** with a cyan square and the **end node** with a red triangle
- Optionally overlays node IDs when `show_labels=True`

In [ ]:
# Visualize network using built in Enviornment class
from operation_safe_passage.environment.environment import Environment

env = Environment(nodes=network["nodes"], output_dir="output")

# Extract start and end node IDs
start_node = network["mission"]["start"]
end_node = network["mission"]["end"]

# Call the built-in visualization function
env.visualize_network(
    start_node=start_node,
    end_node=end_node,
    show_fig=True
)

## Summary

- A hexagonal map was generated from the minimal configuration file.
- The output (`network.json`) contains nodes, edges, and mission start/end identifiers.
- Each node includes metadata such as terrain type, weather conditions, and mine presence.
- The graph visualization provides a quick overview of terrain distribution.

This map can now be used with:
- The Mission Controller for running simulations
- The Gym environment for reinforcement learning experiments